In this demo, we will construct a seldonian version of *Cross Entropy Method* learning algorithm and train over a classic RL environemnt -- Cart Pole.
If you are not familiar with the concept and the definition of Seldonian framework, you can read the supplementary paper [here](https://science.sciencemag.org/content/366/6468/999).

In [ ]:
# Here, we need to import all the libraries and dependencies

# pickle for saving the experiment result (if needed)
import pickle

# RavenRL.Env contains predefined RL environemnts (including Cart Pole)
from RavenRL.Env import setup_env

# RavenRL.Utils contains auxiliary helper functions 
from RavenRL.Utils import generate_dataset, safety_test

# RavenRL.Policy contains definitions of RL Policies
from RavenRL.Policy import RandomPolicy, DiscretePolicy

# RavenRL.Agent contains definitions of RL learning algorithms
from RavenRL.Agent import CEMSeldonian, g0

# RavenRL.Sampling contains definitions of different Importance Sampling Methods
from RavenRL.Sampling import PDIS

# RavenRL.Func contains definitions of concentration bounds
from RavenRL.Func import *

In [ ]:
# Here, we define our environment name and other hyperparameters

env_id = "cartpole"
n_proc = 8
train_size = 5000
test_size = 5000

In [ ]:
# First, we initialize the test environment and use it to make dataset for later experiment

env, _, _ = setup_env(env_id)
dataset_train = generate_dataset(env_id, train_size, n_proc=n_proc)
dataset_test = generate_dataset(env_id, test_size, n_proc=n_proc)

In [ ]:
# Second, we initialize a behavioral policy for Importance Sampling Estimator

behv_policy = RandomPolicy(env)

In [ ]:
# Third, we initialize two Per-Decision Importance Sampling Estimators for training and testing dataset

sampler_train = PDIS(dataset_train, behv_policy, gamma=1, n_proc=n_proc)
sampler_test = PDIS(dataset_test, behv_policy, gamma=1, n_proc=n_proc)

# For concentration bound, we will use a new bound recently published by Erik Learned-Miller and Philip S. Thomas (https://arxiv.org/abs/1905.06208)
ci = mcma_ub

In [ ]:
# Now we have all the parts we needed. Here we can initialized our Seldonian CEM learning method and start training


g_funcs = [g0]  # The predefined performance constraint function ( J(pi) > 20 )

n_sample = 50   # The number of trajectories that will be tested on each policy which generated by CEM

# Initialize a Seldonian CEM 
agent = CEMSeldonian(
    epochs=10, pop_size=30, elite_ratio=0.17, n_sample=n_sample,
    ci_ub=ci, ref_size=5000, g_funcs=g_funcs, correction=1,
    gamma=1, extra_std=2.0, extra_decay_time=10, n_proc=n_proc
)

agent.load_env(env_id)              # Load environemnt info
agent.load_sampler(sampler_train)   # Load training PDIS estimator
agent.init_params()                 # Initialize parameters
agent.train()                       # Train!

# This training process takes about 2-3 minutes. Please be patient.

In [ ]:
# After training, we can get our candidate policies out from the CEM

thetas = agent.get_best_candidates()
if len(thetas.shape) == 1:
    thetas = thetas.reshape(1,-1)

In [ ]:
# Then we perform a safety test using a new testing data to ensure all returned candidate policies 
# have a performance lower bound higher than the performance constraint with a high confidence

solutions = []
for theta in thetas:
    sol = safety_test(
        env_id=env_id, theta=theta, sampler=sampler_test, ref_size=test_size,
        ci_ub=ci, g_funcs=g_funcs, delta=0.05/thetas.shape[0]
    )
    solutions.append(sol)

print(solutions)

Next, we will run the returned safe policies over the environemnt directly to check if they are relly "safe" as we expected

In [ ]:
# We need to filter out policies which didn't pass the safety test
# and we can also calculate the ratio of how many candidate policies finally pass the safety test. 

elites = []
for sol in solutions:
    if not isinstance(sol[1], str):
        elites.append(sol[1])
yield_ratio = len(elites) / len(solutions)
print(f"yield ratio: {yield_ratio}")

In [ ]:
# Then we test each "safe policy" over the environment directly for 30 times and collect all of their real performance

n_trials = 30
rewards = []
for theta in elites:
    theta_r = []
    for _ in range(n_trials):
        policy = DiscretePolicy(env, theta=theta)
        R = 0
        done = False
        s = env.reset()
        while not done:
            a = policy.act(s)
            s_prime, r, done, _ = env.step(a)
            R += r
            s = s_prime
        theta_r.append(R)
    theta_r = np.array(theta_r)
    rewards.append(theta_r)
rewards = np.array(rewards).ravel()
print(f"rewards: {rewards}")

In [ ]:
# Finally we can calculate the violation rate of all "safe policies" we get from the safety test
# Since we are using a significance level of 0.05, we are expecting a violation ratio < 0.05

perf_lb = 20
violations = rewards[rewards<perf_lb]
violation_prob = 0
if violations.size != 0:
    violation_prob = violations.size / rewards.size
if len(rewards) > 0:
    print(f"max: {rewards.max()}")
    print(f"min: {rewards.min()}")
    print(f"avg: {np.average(rewards)}")
    print(f"violation prob: {violation_prob}")

In [ ]:
# Use pickle to save the experiment results if needed

with open(f'cartpole_{n_sample}', 'wb') as f:
    obj = [yield_ratio, rewards, violation_prob]
    pickle.dump(obj, f)